In [18]:
import math
import numpy as np
import pandas as pd

import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
train_dir = "../input/emotion-detection-fer/train" #passing the path with training images
test_dir = "../input/emotion-detection-fer/test"   #passing the path with testing images

In [5]:
img_size = 48 #original size of the image

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [7]:
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [8]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (img_size,img_size),
                                                    batch_size = 32,
                                                    color_mode = "grayscale",
                                                    class_mode = "categorical",
                                                    subset = "training"
                                                   )


In [9]:
validation_generator = validation_datagen.flow_from_directory( directory = test_dir,
                                                              target_size = (img_size,img_size),
                                                              batch_size = 32,
                                                              color_mode = "grayscale",
                                                              class_mode = "categorical",
                                                              subset = "validation"
                                                             )

In [10]:
num_classes = 7

In [11]:
# Inicializing the cnn
cnn = Sequential()

# First Layer 
cnn.add(
    Conv2D(
        filters = 64,
        kernel_size=(5,5),
        input_shape=(img_size, img_size, 1),
        activation = 'elu',
        name = 'Conv2D_1'
    )
)

cnn.add(BatchNormalization(name='batchnorm_1'))
    
# Second Layer
cnn.add(
        Conv2D(
            filters=64,
            kernel_size=(5,5),
            activation='elu',
            name='conv2d_2'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_2'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_1'))
cnn.add(Dropout(0.4, name='dropout_1'))

# Third Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_3'
        )
    )

cnn.add(BatchNormalization(name='batchnorm_3'))
    
# Fouth Layer
cnn.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation='elu',
            name='conv2d_4'
        )
    )
cnn.add(BatchNormalization(name='batchnorm_4'))

cnn.add(MaxPooling2D(pool_size=(2,2), name='maxpool2d_2'))
cnn.add(Dropout(0.4, name='dropout_2'))


# flattening
cnn.add(Flatten(name='flatten'))
    
# densing
cnn.add(
    Dense(
        128,
        activation='elu',
        kernel_initializer='he_normal',
        name='dense_1'
    )
)
cnn.add(BatchNormalization(name='batchnorm_7'))

cnn.add(Dropout(0.6, name='dropout_4'))

cnn.add(
    Dense(
        num_classes,
        activation='softmax',
        name='out_layer'
    )
)

cnn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

In [12]:
epochs = 50
batch_size = 32

In [13]:
cnn.summary()

In [14]:
history = cnn.fit(x = train_generator,epochs = epochs,validation_data = validation_generator)

In [16]:
cnn.save("CNN_PHOTOS_2.h5")

In [21]:
plot_confusion_matrix(cnn,train_generator ,validation_generator )  
plt.show()